In [1]:
import tensorflow as tf
import pandas as pd
import mbuild as mb
import numpy as np
import warnings
warnings.filterwarnings('ignore')

This code will train a very simple neural network model to tell us if a pair of atoms at distance x should be bonded. 

To train the model, we will loop over all pairs of atoms in a given molecule, storing the atomic number of each atom, the distance, and whether the pair of atoms are bonded.  To gather the training data, we will generate a series of molecules from the smiles strings below, as we can trivially load these into mbuild Compounds. 

This will return a pandas data frame as it will make it easier to parse up into training/validation.


In [2]:
smiles_strings = ['C', 'CC', 'CCC', 'CCCC', 'CCCCC', 'CCCCCC', 'CCCCCCCCCCCC',
                  'C1CCCCC1', 'OCCOCCO', 'CO', 'CCO', 'CCCO', 'CCCCO',
                  'C1CNC(CC1CP(=O)(O)O)C(=O)O', 'c1ccccc1',
                 'C1CCCCC1', 'CCCCC=C', 'O=C=O', 'CCN(CC)CC', 'CC(=O)O', 'C1=CC(=CC=C1Cl)Cl']
                  

In [3]:
def generate_data(smiles_strings):
    data_set = []
    for smile in smiles_strings:
        comp1 = mb.load(smile, smiles=True)

        for p1 in comp1.particles():
            for p2 in comp1.particles():
                if p1 != p2:
                    distance = np.linalg.norm(p1.xyz-p2.xyz)

                    # let us exclude any values greater than 0.5 nm
                    # this is still very long for a bond, but we want to make sure we are training the model
                    # with cases where no bonds should exist. 
                    # if we limited it to a much smaller value, say 0.2 nm, our dataset with be about 1/5 the size
                    # and be almost exclusively bonded atoms. 
                    # We basically want the code to understand that no bonds should exist at larger distances
                    if distance < 0.5:
                        if comp1.bond_graph.has_edge(p1,p2):
                            temp_dict = {'A': p1.element.atomic_number, 'B': p2.element.atomic_number, 
                                 'dist': distance*10.0,
                                'bonded': 1}
                            data_set.append(temp_dict)
                        else:
                            temp_dict = {'A': p1.element.atomic_number, 'B': p2.element.atomic_number, 
                                 'dist': distance*10,
                                'bonded': 0}
                            data_set.append(temp_dict)
    return pd.DataFrame(data_set)


In [4]:
data_set = generate_data(smiles_strings)

Split our data such that 75% are training 25% are for validation

In [5]:
train_df = data_set.sample(frac=0.75, random_state=4)
val_df = data_set.drop(train_df.index)


We will then further split this up for training/testing the model, basically splitting the 'bonded' column into Y_train and Y_val.

In [6]:
X_train = train_df.drop('bonded',axis=1)
Y_train = train_df['bonded']


X_val = val_df.drop('bonded',axis=1)
Y_val = val_df['bonded']

In [7]:
input_shape = [X_train.shape[1]]

Set up a sequential model with 3 layers: 1 input layer, 1 hidden layer with 64 units, and 1 output layer.

Since this is basically a classification problem, we will set the output layer to have 2 units (i.e., 2 states, bonded or not bonded) and use softmax activation (i.e.,  the sum of the outputs will be 1 for each pair such that each value in the vector is basically a probability).

In [8]:
model = tf.keras.Sequential([
 
    tf.keras.layers.Dense(units=64, activation='relu',
                          input_shape=input_shape),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                256       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 4,546
Trainable params: 4,546
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', 
              #loss= 'mae',
              loss =tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [10]:
losses = model.fit(X_train, Y_train,
                   validation_data=(X_val, Y_val),
                    
                   # it will use 'batch_size' number
                   # of examples per example
                   batch_size=256,
                   epochs=100,  # total epoch
 
                   )

Epoch 1/100


2023-06-18 01:20:22.083289: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


14/14 [==============================] - 0s 16ms/step - loss: 0.5741 - accuracy: 0.7327 - val_loss: 0.4345 - val_accuracy: 0.8693
Epoch 2/100
14/14 [==============================] - 0s 2ms/step - loss: 0.3446 - accuracy: 0.8756 - val_loss: 0.2851 - val_accuracy: 0.9484
Epoch 3/100
14/14 [==============================] - 0s 2ms/step - loss: 0.2386 - accuracy: 0.8901 - val_loss: 0.2071 - val_accuracy: 0.8827
Epoch 4/100
14/14 [==============================] - 0s 2ms/step - loss: 0.1771 - accuracy: 0.9093 - val_loss: 0.1648 - val_accuracy: 0.9511
Epoch 5/100
14/14 [==============================] - 0s 3ms/step - loss: 0.1400 - accuracy: 0.9766 - val_loss: 0.1386 - val_accuracy: 0.9458
Epoch 6/100
14/14 [==============================] - 0s 2ms/step - loss: 0.1163 - accuracy: 0.9828 - val_loss: 0.1251 - val_accuracy: 0.9849
Epoch 7/100
14/14 [==============================] - 0s 2ms/step - loss: 0.1000 - accuracy: 0.9932 - val_loss: 0.1149 - val_accuracy: 0.9849
Epoch 8/100
14/14 [=====

Test the accuracy and we can see that we are perfect, which I would expect since there isn't much ambiguity.

In [12]:
test_loss, test_acc = model.evaluate(X_val,  Y_val, verbose=2)

print('\nTest accuracy:', test_acc)

36/36 - 0s - loss: 6.6775e-04 - accuracy: 1.0000 - 31ms/epoch - 847us/step

Test accuracy: 1.0


In [13]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [14]:
predictions = probability_model.predict(X_val)


36/36 [==============================] - 0s 591us/step


Let us now use the model.  We'll generate a molecule, remove the bonds, then use the model to add the back in. 

In [15]:
def add_bonds(comp1, prob_model):
        
    particles = [p for p in comp1.particles()]
    
    input_list = []
    pairs = []
    for i in range(0, len(particles)-1):
        for j in range(i+1, len(particles)):
            p1 = particles[i]
            p2 = particles[j]
            
            distance = np.linalg.norm(p1.xyz-p2.xyz)

            if distance < 0.5:
                temp_dict = {'A': p1.element.atomic_number, 'B': p2.element.atomic_number, 
                         'dist': distance*10}
                input_list.append(temp_dict)
                pairs.append((p1, p2))
                
                
    test = pd.DataFrame(input_list)
    
    val = prob_model.predict(test)
    
    for i in range(0, len(val)):
        if np.argmax(val[i]) == 1:
            comp1.add_bond((pairs[i]))
            
    return comp1

In [16]:
original = mb.load('C1CCCCC1CO', smiles=True)
no_bonds = mb.clone(original)

for bond in no_bonds.bonds():
    no_bonds.remove_bond(bond)

In [17]:
no_bonds.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [18]:
mol_with_bonds = add_bonds(no_bonds, probability_model)
mol_with_bonds.visualize()

7/7 [==============================] - 0s 1ms/step


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Let us try something with Cl atoms; we don't have many molecules with Cl in our data set, so not sure how well it will do. 

In [19]:
original = mb.load('C1=CC(=C(C=C1Cl)Cl)Cl', smiles=True)
no_bonds = mb.clone(original)

for bond in no_bonds.bonds():
    no_bonds.remove_bond(bond)
    


In [21]:
mol_with_bonds = add_bonds(no_bonds, probability_model)
mol_with_bonds.visualize()

2/2 [==============================] - 0s 2ms/step


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

It almost works!  This I think is an issue with the fact we have very limited data for Cl-Cl bonds in our data set. Let us can add some more molecules to our dataset in hopes of improving the accuracy.

Since the model is cheap, I'm just going to regenerate it, rather than trying to add more training to it. 

In [22]:
smiles_strings.append('C(Cl)Cl')
smiles_strings.append('CC(Cl)Cl')
smiles_strings.append('CC(Cl)CCl')
smiles_strings.append('CCC(Cl)Cl')
smiles_strings.append('CC(Cl)C(Cl)Cl')

data_set = generate_data(smiles_strings)

train_df = data_set.sample(frac=0.75, random_state=4)
val_df = data_set.drop(train_df.index)

X_train = train_df.drop('bonded',axis=1)
Y_train = train_df['bonded']


X_val = val_df.drop('bonded',axis=1)
Y_val = val_df['bonded']

input_shape = [X_train.shape[1]]

model = tf.keras.Sequential([
 
    tf.keras.layers.Dense(units=64, activation='relu',
                          input_shape=input_shape),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2, activation='softmax')
])

model.compile(optimizer='adam', 
              #loss= 'mae',
              loss =tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

losses = model.fit(X_train, Y_train,
                   validation_data=(X_val, Y_val),
                    
                   # it will use 'batch_size' number
                   # of examples per example
                   batch_size=256,
                   epochs=100,  # total epoch
 
                   )

test_loss, test_acc = model.evaluate(X_val,  Y_val, verbose=2)

print('\nTest accuracy:', test_acc)

probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

Epoch 1/100
15/15 [==============================] - 0s 6ms/step - loss: 0.4020 - accuracy: 0.8633 - val_loss: 0.3150 - val_accuracy: 0.8622
Epoch 2/100
15/15 [==============================] - 0s 2ms/step - loss: 0.2620 - accuracy: 0.8644 - val_loss: 0.2179 - val_accuracy: 0.8719
Epoch 3/100
15/15 [==============================] - 0s 3ms/step - loss: 0.1973 - accuracy: 0.8837 - val_loss: 0.1749 - val_accuracy: 0.9103
Epoch 4/100
15/15 [==============================] - 0s 2ms/step - loss: 0.1656 - accuracy: 0.9043 - val_loss: 0.1521 - val_accuracy: 0.9021
Epoch 5/100
15/15 [==============================] - 0s 2ms/step - loss: 0.1470 - accuracy: 0.9060 - val_loss: 0.1365 - val_accuracy: 0.9462
Epoch 6/100
15/15 [==============================] - 0s 2ms/step - loss: 0.1315 - accuracy: 0.9397 - val_loss: 0.1212 - val_accuracy: 0.9869
Epoch 7/100
15/15 [==============================] - 0s 2ms/step - loss: 0.1182 - accuracy: 0.9821 - val_loss: 0.1080 - val_accuracy: 0.9918
Epoch 8/100
1

In [23]:
original = mb.load('C1=CC(=C(C=C1Cl)Cl)Cl', smiles=True)
no_bonds = mb.clone(original)

for bond in no_bonds.bonds():
    no_bonds.remove_bond(bond)

mol_with_bonds = add_bonds(no_bonds, probability_model)
mol_with_bonds.visualize()

2/2 [==============================] - 0s 991us/step


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Now it works!  I wonder what would happen if we give something we didn't train on at all, say a thiol molecule.  I'd expect there to be no issues with the C-C and C-H bond identification in the molecule, but don't really know what it will do with the C-S.  

I'm guessing it will be able to do some interpolation...we already include C-Cl bonds of length ~1.7 angstroms; the code sees this bond being between numbers 6 and 17. The C-S bond is between 6 and 16, with bond length is ~1.6 angstroms, so it should be within our bonds.

In [24]:
original = mb.load('CCCS', smiles=True)
no_bonds = mb.clone(original)

for bond in no_bonds.bonds():
    no_bonds.remove_bond(bond)

mol_with_bonds = add_bonds(no_bonds, probability_model)
mol_with_bonds.visualize()


3/3 [==============================] - 0s 1ms/step


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

And it seems to work which is great. 

This obviously is only considering isolated molecules.  It could certainly be improved by considering data coming from simulations of bulk materials, where there would be more pairs to consider, but ultimately, this is just a fun demo and not necessarily all that useful. 
